In [1]:
#LB .0114299

import kagglegym
import numpy as np
import pandas as pd
# from sklearn import linear_model as lm
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression


In [2]:
# target = 'y'

# # The "environment" is our interface for code competitions
# env = kagglegym.make()

# # We get our initial observation by calling "reset"
# observation = env.reset()

env = kagglegym.make()
o = env.reset()
#o.train = o.train[:1000]
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = [c for c in o.train.columns if c not in excl]



In [3]:
# # Get the train dataframe
# train = observation.train
# mean_values = train.mean(axis=0)
# train.fillna(mean_values, inplace=True)

train = pd.read_hdf('../input/train.h5')
train = train[col]
d_mean= train.median(axis=0)

train = o.train[col]
n = train.isnull().sum(axis=1)
for c in train.columns:
    train[c + '_nan_'] = pd.isnull(train[c])
    d_mean[c + '_nan_'] = 0
train = train.fillna(d_mean)
train['znull'] = n
n = []



KeyboardInterrupt: 

In [ ]:
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
model1 = rfr.fit(train, o.train['y'])
train = []

In [ ]:
# cols_to_use = ['technical_30', 'technical_20', 'fundamental_11']

# Observed with histograns:
low_y_cut = -0.086093
high_y_cut = 0.093497

y_is_above_cut = (o.train.y > high_y_cut)
y_is_below_cut = (o.train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)

ridge1 = Ridge() ## f11 only
ridge2 = Ridge() ## t30 and f11
ridge3 = Ridge() ## t20 and f11
ridge4 = Ridge() ## t30, t20, f11

train = o.train.loc[y_is_within_cut]

index1 = train.query("technical_30 == 0.0 & technical_20 == 0.0").index
index2 = train.query("technical_30 != 0.0 & technical_20 == 0.0").index
index3 = train.query("technical_30 == 0.0 & technical_20 != 0.0").index
index4 = train.query("technical_30 != 0.0 & technical_20 != 0.0").index


####no fillna in orginal code!
ridge1.fit(train.fillna(d_mean).loc[index1, ["fundamental_11"]].values, train.loc[index1].y)
ridge2.fit(train.fillna(d_mean).loc[index2, ['technical_30', 'fundamental_11']].values, train.loc[index2].y)
ridge3.fit(train.fillna(d_mean).loc[index3, ['technical_20', 'fundamental_11']].values, train.loc[index3].y)
ridge4.fit(train.fillna(d_mean).loc[index4, ['technical_30', 'technical_20', 'fundamental_11']].values, \
           train.loc[index4].y)


# model = Ridge()
# model.fit(np.array(train.loc[y_is_within_cut, cols_to_use].values), train.loc[y_is_within_cut, target])

# ymean_dict = dict(train.groupby(["id"])["y"].mean())
ymedian_dict = dict(o.train.groupby(["id"])["y"].median())


In [ ]:
coeff1 = .65
coeff2 =.95

def get_weighted_y(series):
    id, y = series["id"], series["y"]
    # return 0.95 * y + 0.05 * ymean_dict[id] if id in ymean_dict else y
    return coeff2 * y + (1-coeff2) * ymedian_dict[id] if id in ymedian_dict else y

while True:
    test = o.features[col]
    n = test.isnull().sum(axis=1)
    for c in test.columns:
        test[c + '_nan_'] = pd.isnull(test[c])
    test = test.fillna(d_mean)
    test['znull'] = n
    pred = o.target
    #observation.features.fillna(mean_values, inplace=True)
    # test_x = np.array(observation.features[cols_to_use].values)
    # observation.target.y = model.predict(test_x).clip(low_y_cut, high_y_cut)
    
    index1 = test.query("technical_30 == 0.0 & technical_20 == 0.0").index
    index2 = test.query("technical_30 != 0.0 & technical_20 == 0.0").index
    index3 = test.query("technical_30 == 0.0 & technical_20 != 0.0").index
    index4 = test.query("technical_30 != 0.0 & technical_20 != 0.0").index
    
    if len(index1) > 0:
        pred.loc[index1, 'y'] = ridge1.predict\
        (test.loc[index1, ["fundamental_11"]].values).clip(low_y_cut, high_y_cut)
    if len(index2) > 0:
        pred.loc[index2, 'y'] = ridge2.predict\
        (test.loc[index2, ['technical_30', 'fundamental_11']].values).clip(low_y_cut, high_y_cut)
    if len(index3) > 0:
        pred.loc[index3, 'y'] = ridge3.predict\
        (test.loc[index3, ['technical_20', 'fundamental_11']].values).clip(low_y_cut, high_y_cut)
    if len(index4) > 0:
        pred.loc[index4, 'y'] = ridge4.predict\
        (test.loc[index4, ['technical_30', 'technical_20', 'fundamental_11']].values).clip(low_y_cut, high_y_cut)

    ## weighted y using average value
    pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * coeff1) + \
                pred['y'] * (1-coeff1)
    pred['y'] = pred.apply(get_weighted_y, axis = 1)
    
    #observation.target.fillna(0, inplace=True)
    #target = observation.target
    timestamp = o.features["timestamp"][0]
    if timestamp % 100 == 0:
        print("Timestamp #{}".format(timestamp))
        print(reward)

    o, reward, done, info = env.step(pred)
    if done:
        break
    
print(info)

In [ ]:
def fitInitialScript3(coeff1=.65, coeff2=.95):
    env = kagglegym.make()
    o = env.reset()
    
    def get_weighted_y(series):
        id, y = series["id"], series["y"]
    # return 0.95 * y + 0.05 * ymean_dict[id] if id in ymean_dict else y
        return coeff2 * y + (1-coeff2) * ymedian_dict[id] if id in ymedian_dict else y

    while True:
        test = o.features[col]
        n = test.isnull().sum(axis=1)
        for c in test.columns:
            test[c + '_nan_'] = pd.isnull(test[c])
        test = test.fillna(d_mean)
        test['znull'] = n
        pred = o.target
    #observation.features.fillna(mean_values, inplace=True)
    # test_x = np.array(observation.features[cols_to_use].values)
    # observation.target.y = model.predict(test_x).clip(low_y_cut, high_y_cut)
    
        index1 = test.query("technical_30 == 0.0 & technical_20 == 0.0").index
        index2 = test.query("technical_30 != 0.0 & technical_20 == 0.0").index
        index3 = test.query("technical_30 == 0.0 & technical_20 != 0.0").index
        index4 = test.query("technical_30 != 0.0 & technical_20 != 0.0").index
    
        if len(index1) > 0:
            pred.loc[index1, 'y'] = ridge1.predict\
            (test.loc[index1, ["fundamental_11"]].values).clip(low_y_cut, high_y_cut)
        if len(index2) > 0:
            pred.loc[index2, 'y'] = ridge2.predict\
            (test.loc[index2, ['technical_30', 'fundamental_11']].values).clip(low_y_cut, high_y_cut)
        if len(index3) > 0:
            pred.loc[index3, 'y'] = ridge3.predict\
            (test.loc[index3, ['technical_20', 'fundamental_11']].values).clip(low_y_cut, high_y_cut)
        if len(index4) > 0:
            pred.loc[index4, 'y'] = ridge4.predict\
            (test.loc[index4, ['technical_30', 'technical_20', 'fundamental_11']].values).clip(low_y_cut, high_y_cut)

    ## weighted y using average value
        pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * coeff1) + \
                    pred['y'] * (1-coeff1)
        pred['y'] = pred.apply(get_weighted_y, axis = 1)
    
    #observation.target.fillna(0, inplace=True)
    #target = observation.target
#         timestamp = o.features["timestamp"][0]
#         if timestamp % 100 == 0:
#             print("Timestamp #{}".format(timestamp))
#             print(reward)

        o, reward, done, info = env.step(pred)
        if done:
            break
    
    print(info)

In [ ]:
n=5
x=-.075+np.arange(-n,n+1)/(n)*.01
y=.075+np.arange(-n,n+1)/(n)*.01
w=.65+np.arange(-n,n+1)/(n)*.05
z=.95+np.arange(-n,n+1)/(n)*.05
for i in w:
    fitInitialScript3(i,.95)